Milestone 4

Imports

In [ ]:
!pip install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.0 MB/s eta 0:00:00


Creating Stable Streamlit App (app.py)

In [ ]:
%%writefile app.py
import streamlit as st
from datetime import datetime
import time

# ----------------------------
# PAGE CONFIG
# ----------------------------
st.set_page_config(page_title="Contract Insight Dashboard", layout="wide")

MAX_FILE_SIZE_MB = 2
RISK_ORDER = {"low": 0, "medium": 1, "high": 2, "unknown": 1}

# ----------------------------
# MOCK BACKEND LOGIC
# ----------------------------
def analyze_contract(contract_name, contract_text, tone="executive"):
    if not contract_text.strip():
        return {"error": "Empty contract"}

    time.sleep(1)

    risk = "high" if "penalty" in contract_text.lower() else "low"

    analysis_json = {
        "contract_name": contract_name,
        "overall_risk": risk,
        "key_clauses": ["payment", "termination"],
        "tone": tone
    }

    report = (
        f"This contract has been analyzed in {tone} tone. "
        f"The overall risk level is {risk.upper()}."
    )

    return {
        "analysis_json": analysis_json,
        "report": report,
        "overall_risk": risk
    }

# ----------------------------
# SIDEBAR
# ----------------------------
st.sidebar.title("⚙️ Controls")
st.sidebar.markdown("**Created by:** Aman")

uploaded_files = st.sidebar.file_uploader(
    "Upload Contract Files",
    type=["txt"],
    accept_multiple_files=True
)

# ----------------------------
# MAIN UI
# ----------------------------
st.title("📄 Contract Insight Dashboard")
st.write("Upload contracts and analyze risk levels.")

contracts = []

if uploaded_files:
    for file in uploaded_files:
        size_mb = file.size / (1024 * 1024)
        if size_mb > MAX_FILE_SIZE_MB:
            st.error(f"❌ {file.name} exceeds {MAX_FILE_SIZE_MB}MB")
            continue

        text = file.read().decode("utf-8").strip()
        if not text:
            st.warning(f"⚠️ {file.name} is empty")
            continue

        contracts.append({
            "name": file.name,
            "text": text,
            "chars": len(text)
        })

# ----------------------------
# PREVIEW
# ----------------------------
if contracts:
    st.subheader("📌 Contract Preview")
    for c in contracts:
        with st.expander(f"{c['name']} ({c['chars']} characters)"):
            st.text(c["text"][:1000])

# ----------------------------
# ANALYSIS
# ----------------------------
if st.button("🔍 Analyze Contracts", disabled=not contracts):
    results = []

    with st.spinner("Analyzing contracts..."):
        for c in contracts:
            result = analyze_contract(c["name"], c["text"])
            result["timestamp"] = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")
            result["contract_name"] = c["name"]
            results.append(result)

    results.sort(
        key=lambda x: RISK_ORDER.get(x.get("overall_risk", "unknown"), 1),
        reverse=True
    )

    st.subheader("📊 Summary")
    st.dataframe([
        {
            "Contract": r["contract_name"],
            "Risk": r["overall_risk"],
            "Timestamp": r["timestamp"]
        }
        for r in results
    ])

    st.subheader("📂 Detailed Analysis")
    for r in results:
        with st.expander(f"{r['contract_name']} — {r['overall_risk'].upper()}"):
            tab1, tab2 = st.tabs(["📑 JSON", "📝 Report"])
            with tab1:
                st.json(r["analysis_json"])
            with tab2:
                st.write(r["report"])

# ----------------------------
# FOOTER
# ----------------------------
st.markdown("---")
st.markdown(
    "<center>© 2026 Contract Intelligence System | Built by Aman</center>",
    unsafe_allow_html=True
)


Overwriting app.py


Running STREAMLIT WITH NGROK

In [ ]:
from pyngrok import ngrok
import subprocess
import os

# Set authtoken (required)
ngrok.set_auth_token("38T4vHLNLquFXmIzfD0fPZHkxkJ_5ce2KBYrRKDXxsSxLrJVm")

# Kill any existing tunnels (safety)
ngrok.kill()

# Start Streamlit
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.enableCORS", "false"]
)

# Create public URL
public_url = ngrok.connect(8501)
print("🔗 Open this URL:", public_url)


🔗 Open this URL: NgrokTunnel: "https://uncensured-boughten-quinn.ngrok-free.dev" -> "http://localhost:8501"
